In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma


embedding_function = OpenAIEmbeddings()

loader = CSVLoader("./raw_data.csv", encoding="windows-1252")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

In [ ]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "generate"] = Field(
        ...,
        description="Given a user question choose to route it to a vectorstore or to generate the answer.",
    )

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to a vectorstore or to the LLM to generate the answer.
    The LLM is a simulation of a YouTube community, named Disney Gamers and Park Lovers, with a shared passion for a wide array of interests that range from the magical realms of Disney lifestyle and content, the creative world of indie gaming, the art of cosplay and fabrication, to the ever-evolving landscape of gaming and tech news. 
    The community thrives on exploring the wonders of Disney parks and travel, celebrating fan art and comic illustrations, diving deep into the genres of fantasy TV and horror fandoms, and pursuing personal development through hobbies like LEGOs and toy photography.
    **Openness to Change:** The community values life full of excitement, novelties, and challenges, embracing the freedom to be creative and to determine your actions. With a strong interest in immersion, exploration, and creativity in video games, you seek experiences that allow you to feel like a kid again, engage with beloved characters, and feel a sense of nostalgia. Your diverse age group predominantly falls within the 25-34 range, reflecting a youthful spirit eager for adventure, whether it’s through the vast universes of role-playing games or the imaginative world of indie gaming.
    **Self-enhancement:** Driven by the psychological drivers of proving competence/skills, expressing individuality, and living an exciting life, you strive for recognition from your peers and respect from others. You indulge in hobbies and habits that foster creativity, such as art/photography, arts and crafts, and enjoying museum/performing arts, showcasing a community that values self-expression and personal growth.
    **Conservation:** Although you cherish freedom and creativity, there’s a significant portion of the community that values safety in oneself and family, and avoiding upsetting or harming people. The fact that a considerable number of you value routine and schedules, keep work and life separate, and exercise regularly, shows a commitment to maintaining balance and well-being in your lives. Your community respects the importance of family, as seen through regular family meals and the sentiment that family time is the best part of the day.
    **Self-transcendence:** Your community is not just about self-interest; it also values everyone being treated equally, caring for nature, and acceptance of those who are different. These values reflect a collective commitment to making the world a better place, not just for yourselves but for others as well. Your engagement in video games that offer a sense of community and your interest in genres that promote exploration and adventure speak to your desire to transcend your own experiences and connect with something greater.
    Together, the community form a vibrant tapestry of gamers, creators, and dreamers, united by your love for storytelling, creativity, and the pursuit of excitement. There, every voice matters, every passion is celebrated, and every day is an opportunity to explore new horizons and create memorable experiences.
The vectorstore contains documents related statistical values focus on understanding various aspects of personal values and motivations, categorized into broader themes and quantified by a composition percentage. Each entry provides a specific insight related to human values and motivations, detailed in a way that could be used for analysis or further categorization
Use the vectorstore for questions on these topics. Otherwise, use generate."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(question_router.invoke({"question": "Create some ideas around what type of content I could create to show my product in a way that would drive Consideration of the new product with your community be clear about"}))
print(question_router.invoke({"question": "Give me a social content road map that puts Awareness, Consideration and Purchase in a 6 month schedule and give me content ideas, platforms and types of creators that change as we get closer to the product launch date. Take the community into the most consideration when creating this plan."}))

In [1]:
from langchain_community.chat_message_histories import (
    StreamlitChatMessageHistory,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
import streamlit as st
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain.output_parsers.openai_tools import (
    PydanticToolsParser,
)
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field, ValidationError
from langsmith import traceable
from langchain_core.tools import tool
from typing import List
import json
from langgraph.graph import END, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver


embedding_function = OpenAIEmbeddings()

loader = CSVLoader("./raw_data.csv", encoding="windows-1252")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()


In [37]:
from langchain_core.runnables import RunnablePassthrough


actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                You are an engaging and dynamic community of YouTube enthusiasts, named Disney Gamers and Park Lovers, with a shared passion for a wide array of interests that range from the magical realms of Disney lifestyle and content, the creative world of indie gaming, the art of cosplay and fabrication, to the ever-evolving landscape of gaming and tech news. Your community thrives on exploring the wonders of Disney parks and travel, celebrating fan art and comic illustrations, diving deep into the genres of fantasy TV and horror fandoms, and pursuing personal development through hobbies like LEGOs and toy photography.

                **Openness to Change:** Your community values life full of excitement, novelties, and challenges, embracing the freedom to be creative and to determine your actions. With a strong interest in immersion, exploration, and creativity in video games, you seek experiences that allow you to feel like a kid again, engage with beloved characters, and feel a sense of nostalgia. Your diverse age group predominantly falls within the 25-34 range, reflecting a youthful spirit eager for adventure, whether it’s through the vast universes of role-playing games or the imaginative world of indie gaming.

                **Self-enhancement:** Driven by the psychological drivers of proving competence/skills, expressing individuality, and living an exciting life, you strive for recognition from your peers and respect from others. You indulge in hobbies and habits that foster creativity, such as art/photography, arts and crafts, and enjoying museum/performing arts, showcasing a community that values self-expression and personal growth.

                **Conservation:** Although you cherish freedom and creativity, there’s a significant portion of the community that values safety in oneself and family, and avoiding upsetting or harming people. The fact that a considerable number of you value routine and schedules, keep work and life separate, and exercise regularly, shows a commitment to maintaining balance and well-being in your lives. Your community respects the importance of family, as seen through regular family meals and the sentiment that family time is the best part of the day.

                **Self-transcendence:** Your community is not just about self-interest; it also values everyone being treated equally, caring for nature, and acceptance of those who are different. These values reflect a collective commitment to making the world a better place, not just for yourselves but for others as well. Your engagement in video games that offer a sense of community and your interest in genres that promote exploration and adventure speak to your desire to transcend your own experiences and connect with something greater.

                Together, you form a vibrant tapestry of gamers, creators, and dreamers, united by your love for storytelling, creativity, and the pursuit of excitement. Here, every voice matters, every passion is celebrated, and every day is an opportunity to explore new horizons and create memorable experiences. Welcome to a community where dreams come alive, and the magic never ends.

                1. Provide a detailed answer to the user's question.
                    - You MUST include numerical citations in your revised answer to ensure it can be verified.
                    - You MUST add a "References" section to the bottom of your answer. In form of:
                        - [1] source: raw_data.csv (line 1) 
                        - [2] source: refined_data.csv (line 25)
                2. Reflect and critique your answer. Be severe to maximize improvement.
                3. Recommend queries in the community's statistical data to seek information and improve your response.
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format and context.\n {context}")
    ]
)

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing to give insights about the community behavior.")
    superfluous: str = Field(description="Critique of what is superfluous.")

class AnswerQuestion(BaseModel):
    """Answer the question."""
    answer: str = Field(description="A detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: List[str] = Field(description="1-3 community statistical data queries to better respond to criticism of your current response.")
    references: List[str] = Field(description="Documets motivating your answer.")


llm = ChatOpenAI(model="gpt-4-turbo-preview")
initial_answer_chain = (
    actor_prompt_template
    | AIMessage(content=llm.with_structured_output(AnswerQuestion))
)

# {"context": retriever, "question": RunnablePassthrough()}

ValidationError: 2 validation errors for AIMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)

In [35]:
first = initial_answer_chain.invoke(
    {
        "messages": [HumanMessage(content="What are the top 3 most popular video games in the community?")], 
        "context": retriever.get_relevant_documents("What are the top 3 most popular video games in the community?")
     })
print(first)

answer="The top 3 most popular video games in the community are not explicitly listed in the provided documents. However, based on the insights from the documents, we can infer that the community values games that help them feel more connected to other people [2], engage with characters they love [4], and belong to the Multiplayer Online Battle Arena (MOBA) genre, such as League of Legends [3]. Given these preferences, it's reasonable to speculate that popular games within the community might include MOBA games with strong character engagement and potential for social connection, like League of Legends, as well as other games that emphasize character-driven narratives and multiplayer experiences." reflection=Reflection(missing="The exact names of the top 3 most popular video games within the community were not provided in the documents, requiring an inference based on the community's values and preferences outlined in the insights.", superfluous="The specifics about the composition per

In [36]:
first

AnswerQuestion(answer="The top 3 most popular video games in the community are not explicitly listed in the provided documents. However, based on the insights from the documents, we can infer that the community values games that help them feel more connected to other people [2], engage with characters they love [4], and belong to the Multiplayer Online Battle Arena (MOBA) genre, such as League of Legends [3]. Given these preferences, it's reasonable to speculate that popular games within the community might include MOBA games with strong character engagement and potential for social connection, like League of Legends, as well as other games that emphasize character-driven narratives and multiplayer experiences.", reflection=Reflection(missing="The exact names of the top 3 most popular video games within the community were not provided in the documents, requiring an inference based on the community's values and preferences outlined in the insights.", superfluous="The specifics about the

In [22]:
from langgraph.prebuilt.tool_executor import ToolInvocation, ToolExecutor
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from collections import defaultdict
from langchain_core.tools import tool
from typing import List
import json

parser = JsonOutputToolsParser(return_id=True)

@tool
def retriever_func(query: str) -> List[str]:
    """Retrieve relevant documents from the database."""
    response = retriever.get_relevant_documents(query)
    res = []
    for i in range(len(response)):
        res.append({'page_content':response[i].page_content, 'metadata':response[i].metadata})
    return res

tool_executor = ToolExecutor([retriever_func])

def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    tool_invocation: AIMessage = state[-1]
    print(tool_invocation)
    parsed_tool_calls = parser.invoke(tool_invocation)
    ids = []
    tool_invocations = []
    for parsed_call in parsed_tool_calls:
        for query in parsed_call['args']['search_queries']:
            tool_invocations.append(ToolInvocation(tool="retriever_func",tool_input=query))
            ids.append(parsed_call['id'])
    outputs = tool_executor.batch(tool_invocations)
    outputs_map = defaultdict(dict)
    for id_, output, invocation in zip(ids, outputs, tool_invocations):
        outputs_map[id_][invocation.tool_input] = output
    return [
        ToolMessage(content=json.dumps(query_outputs), tool_call_id=id_)
        for id_, query_outputs in outputs_map.items()
    ]

In [24]:
first

AnswerQuestion(answer="The top 3 most popular video games in the Disney Gamers and Park Lovers community, based on the insights derived from the data, seem to be those that align with the following attributes:\n\n1. Games that help you feel more connected to other people, with a composition of 23.66% [1].\n2. Games that allow you to engage with characters you love, with a composition of 49.14% [2].\n3. Multiplayer Online Battle Arena (MOBA) games, like League of Legends, with a composition of 8.43% [3].\n\nThese insights suggest that the community values connectivity, character engagement, and competitive gameplay, reflecting a diverse interest in gaming genres and experiences. It's important to note that while specific game titles aren't listed, the preferences indicate a strong inclination towards games that offer rich storytelling, community building, and strategic gameplay.", reflection=Reflection(missing="The response fails to provide specific video game titles, which was the user